<a href="https://colab.research.google.com/github/jduell12/DS-Unit-2-Applied-Modeling/blob/master/C_Define_ML_prob_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

# Choose your target. Which column in your tabular dataset will you predict?

- Predict the probability of complication to MI
  - New Column = Complication_Arose
    - combination of column 113-123
      - 'Atrial_Fibrillation_Outcome', 'Supraventricular_Tachycardia_Outcome',
       'Ventricular_Tachycardia_Outcome', 'Ventricular_Fibrillation_Outcome',
       'Third_Degree_AV_Block_Outcome', 'Pulmunary_Edema_Outcome',
       'Myocardial_Rupture_Outcome', 'Dressler_Syndrome_Outcome',
       'CHD_Outcome', 'Relapse_Myocardial_Infarction',
       'Post-infarction_Outcome'

- Predict probability of deaht
  - Column 124 = LET_IS = Leathal_Cause_Outcome 

# Is your problem regression or classification?

Classification Problem

# How is your target distributed?

* Classification: How many classes? Are the classes imbalanced?
 * Binary classification: 2 classes 


# Choose your evaluation metric(s)
* Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?


# Begin to clean and explore your data.

In [22]:
%%capture
import sys

data_set_paths = ['https://raw.githubusercontent.com/jduell12/DS-Unit-2-Applied-Modeling/master/data/covid/united_states_covid19_cases_deaths_and_testing_by_state.csv',
                  'https://raw.githubusercontent.com/jduell12/DS-Unit2-PortfolioProj/main/MI.csv']

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = data_set_paths[1]
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [136]:
from category_encoders import OneHotEncoder, OrdinalEncoder
from pandas_profiling import ProfileReport 
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [250]:
#CHD = coronary heart disease
#Thyrotoxicosis means an excess of thyroid hormone in the body
#IC --> At admission to Intensive Care Unit
# ESR = Erythrocyte sedimentation rate

def wrangle (path):
  df = pd.read_csv(path)
  df = df.rename(columns={
    "INF_ANAM": 'Quantity', 'STENOK_AN':'Year_Last_angina', 'FK_STENOK': 'Functional_Class_Last_Year',
    "IBS_POST": "CHD_Before_Admission", "IBS_NASL": 'CHD_Heredity', "GB": 'Stage_Hypertension',
    "SIM_GIPERT": 'Symptomatic_Hypertension', "DLIT_AG": "Duration_aterial_hypertension_years",
    "ZSN_A": "Stage_Heart_Failure", "nr_11": 'Arrythmia', "nr_01": "Premature_Atrial_Contractions",
    "nr_02": "Premature_Ventricular_Contractions", "nr_03": "Sudden_Atrial_Fibrillation", "nr_04": "Persistent_Atrial_Fibrillation",
    "nr_07": "Ventricular_Fibrillation", "nr_08": "Sudden_Ventricular_Tachycardia", "np_01": "First_Degree_AV_Block",
    "np_04": "Third_Degree_AV_Block", "np_05": "Left_Bundle_Branch_Block_Anterior_Branch", "np_07": "Incomplete_Left_Bundle_Branch_Block", "np_08": "Complete_Left_Bundle_Branch_Block",
    "np_09": "Incomplete_Right_Bundle_Block", "np_10": "Complete_Right_Bundle_Block", "endocr_01": "Diabetes", "endocr_02": "Obesity", 
    "endocr_03": "Thyrotoxicosis", "zab_leg_01": "Chronic_Bronchitis", "zab_leg_02": "Obstructive_Chronic_Bronchitis",
    "zab_leg_03": "Bronchial_Asthma", "zab_leg_04": "Chronic_Pneumonia", "zab_leg_06": "Pulmonary_Tuberculosis",
    "S_AD_KBRIG": "Systolic_BP_ER_mmHg", "D_AD_KBRIG": "Diastolic_BP_ER_mmHg", "S_AD_ORIT": "Systolic_BP_IC_mmHg",
    "D_AD_ORIT": "Diastolic_BP_IC_mmHg", "O_L_POST": "Pulmonary_Edema_IC", "K_SH_POST": "Cardiogenic_Shock_IC",
    "MP_TP_POST": "Sudden_Atrial_Fibrillation_IC", "SVT_POST": "Sudden_Supraventricular_Tachycardia_IC",
    "GT_POST": "Sudden_Ventricular_Tachycardia_IC", "FIB_G_POST": "Ventricular_Fibrillation_IC", "ant_im": "Presence_Anterior_Myocardial_Infarction", 
    "lat_im": "Presence_Lateral_Myocardial_Infarction", "inf_im": "Presence_Inferior_Myocardial_Infarction", "post_im": "Presence_Posterior_Myocardial_Infarction",
    "IM_PG_P": "Presence_Right_Ventricular_Myocardial_Infarction", "ritm_ecg_p_01": "ECG_Rhythm_Sinus_Admission", "ritm_ecg_p_02": "ECG_Rhythm_Atrial_Fibrillation_Admission", 
    "ritm_ecg_p_04": "ECG_Rhythm_Atrial_Admission", "ritm_ecg_p_06": "ECG_Rhythm_Idioventricular_Admission", "ritm_ecg_p_0": "ECG_Rhythm_Tachycardia_Admission",
    "ritm_ecg_p_08": "ECG_Rhythm_Bradycardia_Admission", "n_r_ecg_p_01": "Premature_Atrial_Contractions_Admission", "n_r_ecg_p_02": "Frequent_Premature_Atrial_Contractions_Admission",
    "n_r_ecg_p_03": "Premature_Ventricular_Contractions_Admission", "n_r_ecg_p_04": "Frequent_Premature_Ventricular_Contractions_Admission", "n_r_ecg_p_05": "Suddent_Atrial_Fibrillation_Admission", 
    "n_r_ecg_p_06": "Persistent_Atrial_Fibrillation_Admission", "n_r_ecg_p_08": "Sudden_Supraventricular_Tachycardia_Admission", "n_r_ecg_p_09": "Sudden_Ventricular_Tachycardia_Admission",
    "n_r_ecg_p_10": "Ventricular_Fibrillation_Admission", "n_p_ecg_p_01": "Sinoatrial_Block_Admission", "n_p_ecg_p_03": "First_Degree_AV_Admission",
    "n_p_ecg_p_04": "Type_1_Second_Degree_AV_Block_Admission", "n_p_ecg_p_05": "Type_2_Second_Degree_AV_Block_Admission",
    "n_p_ecg_p_06": "Third_Degree_AV_Block_Admission", "n_p_ecg_p_07": "Left_Bundle_Branch_Block_Anterior_Admission", 
    "n_p_ecg_p_08": "Left_Bundle_Branch_Block_Posterior_Admission", "n_p_ecg_p_09": "Incomplete_Left_Bundle_Branch_Block_Admission",
    "n_p_ecg_p_10": "Complete_Left_Bundle_Branch_Block_Admission", "n_p_ecg_p_11": "Incomplete_Right_Bundle_Branch_Block_Admission", 
    "n_p_ecg_p_12": "Complete_Right_Bundle_Branch_Block_Admission", "fibr_ter_01": "Fibrinolytic_Therapy_Celiasum_750k_IU", 
    "fibr_ter_02": "Fibrinolytic_Therapy_Celiasum_1m_IU", "fibr_ter_03": "Fibrinolytic_Therapy_Celiasum_3m_IU", 
    "fibr_ter_05": "Fibrinolytic_Therapy_Streptase", "fibr_ter_06": "Fibrinolytic_Therapy_Celiasum_500k_IU",
    "fibr_ter_07": "Fibrinolytic_Therapy_Celiasum_250k_IU", "fibr_ter_08": "Fibrinolytic_Therapy_Streptodecase_1.5m_IU", 
    "GIPO_K": "Hypokalemia", "K_BLOOD": "Serum_Potassium_mmol/L", "GIPER_NA": "Serum_Sodiam_more_150mmol/L", 
    "Na_BLOOD": "NA_BLOOD_mmol/L", "ALT_BLOOD": "ALT_BLOOD_IU/L", "AST_BLOOD": "AST_BLOOD_IU/L", "KFK_BLOOD": "CPK_BLOOD_IU/L",
    "L_BLOOD": "White_Cell_BLOOD_billions/L", "ROE": "ESR_mm", "TIME_B_S": "Time_To_Hospital", "R_AB_1_n": "Relapse_Pain_Hospital_Day1",
    "R_AB_2_n": "Relapse_Pain_Hospital_Day2", "R_AB_3_n": "Relapse_Pain_Hospital_Day3", "NA_KB": "Opiod_ER", "NOT_NA_KB": "NSAID_ER",
    "LID_KB": "Lidocaine_ER", "NITR_S": "Liquid_Nitrates_ICU", "NA_R_1_n": "Opiod_ICU_Day1", "NA_R_2_n": "Opiod_ICU_Day2", 
    "NA_R_3_n": "Opiod_ICU_Day3", "NOT_NA_1_n": "NSAIDS_ICU_Day1", "NOT_NA_2_n": "NSAIDS_ICU_Day2", "NOT_NA_3_n": "NSAIDS_ICU_Day3",
    "LID_S_n": "Lidocain_ICU", "B_BLOK_S_n": "Beta_Blocker_ICU", "ANT_CA_S_n": "CA_Channel_Blocker_ICU", "GEPAR_S_n": "Heparin_ICU", 
    "ASP_S_n": "Acetylsalicylic_Acid_ICU", "TIKL_S_n": "Ticlid_ICU", "TRENT_S_n": "Trental_ICU", "FIBR_PREDS": "Atrial_Fibrillation_Outcome", "PREDS_TAH": "Supraventricular_Tachycardia_Outcome",
    "JELUD_TAH": "Ventricular_Tachycardia_Outcome", "FIBR_JELUD": "Ventricular_Fibrillation_Outcome", "A_V_BLOK": "Third_Degree_AV_Block_Outcome",
    "OTEK_LANC": "Pulmunary_Edema_Outcome", "RAZRIV": "Myocardial_Rupture_Outcome", "DRESSLER": "Dressler_Syndrome_Outcome", 
    "ZSN": "CHD_Outcome", "REC_IM": "Relapse_Myocardial_Infarction", "P_IM_STEN": "Post-infarction_Outcome",
    "LET_IS": "Lethal_Cause_Outcome"
    })
  columns = ['Atrial_Fibrillation_Outcome', 'Supraventricular_Tachycardia_Outcome', 'Ventricular_Tachycardia_Outcome', 'Ventricular_Fibrillation_Outcome', 'Third_Degree_AV_Block_Outcome', 'Pulmunary_Edema_Outcome', 'Myocardial_Rupture_Outcome', 'Dressler_Syndrome_Outcome', 'CHD_Outcome', 'Relapse_Myocardial_Infarction', 'Post-infarction_Outcome']
  #create outcome column which is combination of all outcome columns 
  df.loc[(df[columns[0]] > 0) | (df[columns[1]] > 0) | (df[columns[2]] > 0) | (df[columns[3]] > 0) | (df[columns[4]] > 0) | (df[columns[5]] > 0) | (df[columns[6]] > 0) | (df[columns[7]] > 0) | (df[columns[8]] > 0) | (df[columns[9]] > 0) | (df[columns[10]] > 0), 'Complication'] = 1 
  df.loc[(df[columns[0]] == 0) & (df[columns[1]] == 0) & (df[columns[2]] == 0) & (df[columns[3]] == 0) & (df[columns[4]] == 0) & (df[columns[5]] == 0) & (df[columns[6]] == 0) & (df[columns[7]] == 0) & (df[columns[8]] == 0) & (df[columns[9]] == 0) & (df[columns[10]] == 0), 'Complication'] = 0
  #drop unneeded outcome columns 
  df.drop(columns=columns, inplace=True)
  #fill na
  df = df.fillna(0)

  #change dtypes from float to int 
  int_cols = ['ID', 'AGE', 'SEX', 'Quantity', 'Year_Last_angina',
       'Functional_Class_Last_Year', 'CHD_Before_Admission', 'CHD_Heredity',
       'Stage_Hypertension', 'Symptomatic_Hypertension',
       'Duration_aterial_hypertension_years', 'Stage_Heart_Failure',
       'Arrythmia', 'Premature_Atrial_Contractions',
       'Premature_Ventricular_Contractions', 'Sudden_Atrial_Fibrillation',
       'Persistent_Atrial_Fibrillation', 'Ventricular_Fibrillation',
       'Sudden_Ventricular_Tachycardia', 'First_Degree_AV_Block',
       'Third_Degree_AV_Block', 'Left_Bundle_Branch_Block_Anterior_Branch',
       'Incomplete_Left_Bundle_Branch_Block',
       'Complete_Left_Bundle_Branch_Block', 'Incomplete_Right_Bundle_Block',
       'Complete_Right_Bundle_Block', 'Diabetes', 'Obesity', 'Thyrotoxicosis',
       'Chronic_Bronchitis', 'Obstructive_Chronic_Bronchitis',
       'Bronchial_Asthma', 'Chronic_Pneumonia', 'Pulmonary_Tuberculosis', 
       'Cardiogenic_Shock_IC', 'Sudden_Atrial_Fibrillation_IC',
       'Sudden_Supraventricular_Tachycardia_IC',
       'Sudden_Ventricular_Tachycardia_IC', 'Ventricular_Fibrillation_IC',
       'Presence_Anterior_Myocardial_Infarction',
       'Presence_Lateral_Myocardial_Infarction',
       'Presence_Inferior_Myocardial_Infarction',
       'Presence_Posterior_Myocardial_Infarction',
       'Presence_Right_Ventricular_Myocardial_Infarction',
       'ECG_Rhythm_Sinus_Admission',
       'ECG_Rhythm_Atrial_Fibrillation_Admission',
       'ECG_Rhythm_Atrial_Admission', 'ECG_Rhythm_Idioventricular_Admission',
       'ritm_ecg_p_07', 'ECG_Rhythm_Bradycardia_Admission',
       'Premature_Atrial_Contractions_Admission',
       'Frequent_Premature_Atrial_Contractions_Admission',
       'Premature_Ventricular_Contractions_Admission',
       'Frequent_Premature_Ventricular_Contractions_Admission',
       'Suddent_Atrial_Fibrillation_Admission',
       'Persistent_Atrial_Fibrillation_Admission',
       'Sudden_Supraventricular_Tachycardia_Admission',
       'Sudden_Ventricular_Tachycardia_Admission',
       'Ventricular_Fibrillation_Admission', 'Sinoatrial_Block_Admission',
       'First_Degree_AV_Admission', 'Type_1_Second_Degree_AV_Block_Admission',
       'Type_2_Second_Degree_AV_Block_Admission',
       'Third_Degree_AV_Block_Admission',
       'Left_Bundle_Branch_Block_Anterior_Admission',
       'Left_Bundle_Branch_Block_Posterior_Admission',
       'Incomplete_Left_Bundle_Branch_Block_Admission',
       'Complete_Left_Bundle_Branch_Block_Admission',
       'Incomplete_Right_Bundle_Branch_Block_Admission',
       'Complete_Right_Bundle_Branch_Block_Admission',
       'Fibrinolytic_Therapy_Celiasum_750k_IU',
       'Fibrinolytic_Therapy_Celiasum_1m_IU',
       'Fibrinolytic_Therapy_Celiasum_3m_IU', 'Fibrinolytic_Therapy_Streptase',
       'Fibrinolytic_Therapy_Celiasum_500k_IU',
       'Fibrinolytic_Therapy_Celiasum_250k_IU',
       'Fibrinolytic_Therapy_Streptodecase_1.5m_IU', 'Hypokalemia','Serum_Sodiam_more_150mmol/L',
       'Time_To_Hospital', 'Relapse_Pain_Hospital_Day1',
       'Relapse_Pain_Hospital_Day2', 'Relapse_Pain_Hospital_Day3', 'Opiod_ER',
       'NSAID_ER', 'Lidocaine_ER', 'Liquid_Nitrates_ICU', 'Opiod_ICU_Day1',
       'Opiod_ICU_Day2', 'Opiod_ICU_Day3', 'NSAIDS_ICU_Day1',
       'NSAIDS_ICU_Day2', 'NSAIDS_ICU_Day3', 'Lidocain_ICU',
       'Beta_Blocker_ICU', 'CA_Channel_Blocker_ICU', 'Heparin_ICU',
       'Acetylsalicylic_Acid_ICU', 'Ticlid_ICU', 'Trental_ICU',
       'Lethal_Cause_Outcome', 'Complication','Systolic_BP_ER_mmHg', 'Diastolic_BP_ER_mmHg', 'Pulmonary_Edema_IC',
       'Systolic_BP_IC_mmHg', 'Diastolic_BP_IC_mmHg', 'ESR_mm'
       ]
  for col in int_cols:
    if df[col].dtype == 'float64':
      df [col] = df[col].apply(int)

  df = df.set_index(df.columns[0])
  return df

In [251]:
df = wrangle(DATA_PATH)

In [252]:
df.shape

(1700, 113)

In [259]:
df.describe()

,AGE,SEX,Quantity,Year_Last_angina,Functional_Class_Last_Year,CHD_Before_Admission,CHD_Heredity,Stage_Hypertension,Symptomatic_Hypertension,Duration_aterial_hypertension_years,Stage_Heart_Failure,Arrythmia,Premature_Atrial_Contractions,Premature_Ventricular_Contractions,Sudden_Atrial_Fibrillation,Persistent_Atrial_Fibrillation,Ventricular_Fibrillation,Sudden_Ventricular_Tachycardia,First_Degree_AV_Block,Third_Degree_AV_Block,Left_Bundle_Branch_Block_Anterior_Branch,Incomplete_Left_Bundle_Branch_Block,Complete_Left_Bundle_Branch_Block,Incomplete_Right_Bundle_Block,Complete_Right_Bundle_Block,Diabetes,Obesity,Thyrotoxicosis,Chronic_Bronchitis,Obstructive_Chronic_Bronchitis,Bronchial_Asthma,Chronic_Pneumonia,Pulmonary_Tuberculosis,Systolic_BP_ER_mmHg,Diastolic_BP_ER_mmHg,Systolic_BP_IC_mmHg,Diastolic_BP_IC_mmHg,Pulmonary_Edema_IC,Cardiogenic_Shock_IC,Sudden_Atrial_Fibrillation_IC,...,Complete_Right_Bundle_Branch_Block_Admission,Fibrinolytic_Therapy_Celiasum_750k_IU,Fibrinolytic_Therapy_Celiasum_1m_IU,Fibrinolytic_Therapy_Celiasum_3m_IU,Fibrinolytic_Therapy_Streptase,Fibrinolytic_Therapy_Celiasum_500k_IU,Fibrinolytic_Therapy_Celiasum_250k_IU,Fibrinolytic_Therapy_Streptodecase_1.5m_IU,Hypokalemia,Serum_Potassium_mmol/L,Serum_Sodiam_more_150mmol/L,NA_BLOOD,ALT_BLOOD_IU/L,AST_BLOOD_IU/L,CPK_BLOOD_IU/L,White_Cell_BLOOD_billions/L,ESR_mm,Time_To_Hospital,Relapse_Pain_Hospital_Day1,Relapse_Pain_Hospital_Day2,Relapse_Pain_Hospital_Day3,Opiod_ER,NSAID_ER,Lidocaine_ER,Liquid_Nitrates_ICU,Opiod_ICU_Day1,Opiod_ICU_Day2,Opiod_ICU_Day3,NSAIDS_ICU_Day1,NSAIDS_ICU_Day2,NSAIDS_ICU_Day3,Lidocain_ICU,Beta_Blocker_ICU,CA_Channel_Blocker_ICU,Heparin_ICU,Acetylsalicylic_Acid_ICU,Ticlid_ICU,Trental_ICU,Lethal_Cause_Outcome,Complication
count,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,...,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.00000
mean,61.565882,0.626471,0.553529,2.171765,1.153529,1.125882,0.015882,1.385882,0.033529,2.852941,0.188235,0.024706,0.002353,0.011176,0.020588,0.017059,0.000588,0.002353,0.001176,0.001765,0.006471,0.000588,0.003529,0.001176,0.001765,0.134118,0.024706,0.007647,0.078824,0.071176,0.021765,0.005294,0.012941,50.252941,29.876471,113.450000,69.752941,0.064706,0.027059,0.067059,...,0.045882,0.007647,0.009412,0.040000,0.002353,0.005294,0.003529,0.001176,0.314118,3.276706,0.017647,106.429412,0.401024,0.219506,0.004706,8.137112,11.839412,4.337059,0.312941,0.131765,0.071765,0.363529,0.412353,0.232941,0.114706,0.483529,0.088235,0.054118,0.330000,0.105882,0.078235,0.281765,0.126471,0.661765,0.707647,0.736471,0.017647,0.200588,0.477059,0.55000
std,12.005042,0.483883,0.836248,2.428718,1.047132,0.813751,0.125057,1.090615,0.180067,3.096946,0.649065,0.155273,0.048464,0.105157,0.142043,0.129529,0.024254,0.048464,0.034290,0.041984,0.080203,0.024254,0.059321,0.034290,0.041984,0.340879,0.155273,0.087138,0.269542,0.257195,0.145957,0.072589,0.113054,69.327643,41.024792,56.814034,34.496259,0.246079,0.162303,0.250197,...,0.209291,0.087138,0.096585,0.196017,0.048464,0.072589,0.059321,0.034290,0.464300,1.855662,0.131704,56.927597,0.396455,0.208813,0.107303,3.996382,11.462074,3.022952,0.631485,0.412064

In [254]:
df.dtypes

AGE                           int64
SEX                           int64
Quantity                      int64
Year_Last_angina              int64
Functional_Class_Last_Year    int64
                              ...  
Acetylsalicylic_Acid_ICU      int64
Ticlid_ICU                    int64
Trental_ICU                   int64
Lethal_Cause_Outcome          int64
Complication                  int64
Length: 113, dtype: object

In [256]:
df.head()

,AGE,SEX,Quantity,Year_Last_angina,Functional_Class_Last_Year,CHD_Before_Admission,CHD_Heredity,Stage_Hypertension,Symptomatic_Hypertension,Duration_aterial_hypertension_years,Stage_Heart_Failure,Arrythmia,Premature_Atrial_Contractions,Premature_Ventricular_Contractions,Sudden_Atrial_Fibrillation,Persistent_Atrial_Fibrillation,Ventricular_Fibrillation,Sudden_Ventricular_Tachycardia,First_Degree_AV_Block,Third_Degree_AV_Block,Left_Bundle_Branch_Block_Anterior_Branch,Incomplete_Left_Bundle_Branch_Block,Complete_Left_Bundle_Branch_Block,Incomplete_Right_Bundle_Block,Complete_Right_Bundle_Block,Diabetes,Obesity,Thyrotoxicosis,Chronic_Bronchitis,Obstructive_Chronic_Bronchitis,Bronchial_Asthma,Chronic_Pneumonia,Pulmonary_Tuberculosis,Systolic_BP_ER_mmHg,Diastolic_BP_ER_mmHg,Systolic_BP_IC_mmHg,Diastolic_BP_IC_mmHg,Pulmonary_Edema_IC,Cardiogenic_Shock_IC,Sudden_Atrial_Fibrillation_IC,...,Complete_Right_Bundle_Branch_Block_Admission,Fibrinolytic_Therapy_Celiasum_750k_IU,Fibrinolytic_Therapy_Celiasum_1m_IU,Fibrinolytic_Therapy_Celiasum_3m_IU,Fibrinolytic_Therapy_Streptase,Fibrinolytic_Therapy_Celiasum_500k_IU,Fibrinolytic_Therapy_Celiasum_250k_IU,Fibrinolytic_Therapy_Streptodecase_1.5m_IU,Hypokalemia,Serum_Potassium_mmol/L,Serum_Sodiam_more_150mmol/L,NA_BLOOD,ALT_BLOOD_IU/L,AST_BLOOD_IU/L,CPK_BLOOD_IU/L,White_Cell_BLOOD_billions/L,ESR_mm,Time_To_Hospital,Relapse_Pain_Hospital_Day1,Relapse_Pain_Hospital_Day2,Relapse_Pain_Hospital_Day3,Opiod_ER,NSAID_ER,Lidocaine_ER,Liquid_Nitrates_ICU,Opiod_ICU_Day1,Opiod_ICU_Day2,Opiod_ICU_Day3,NSAIDS_ICU_Day1,NSAIDS_ICU_Day2,NSAIDS_ICU_Day3,Lidocain_ICU,Beta_Blocker_ICU,CA_Channel_Blocker_ICU,Heparin_ICU,Acetylsalicylic_Acid_ICU,Ticlid_ICU,Trental_ICU,Lethal_Cause_Outcome,Complication
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,77,1,2,1,1,2,0,3,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,180,100,0,0,0,...,0,0,0,0,0,0,0,0,0,4.7,0,138.0,0.00,0.00,0.0,8.0,16,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0
2,55,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,90,0,0,0,...,0,0,0,0,0,0,0,0,1,3.5,0,132.0,0.38,0.18,0.0,7.8,3,2,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,1,1,0,1,0,0
3,52,1,0,0,0,2,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,100,180,100,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0,0,132.0,0.30,0.11,0.0,10.8,0,3,3,0,0,1,1,1,0,1,0,0,3,2,2,1,1,0,1,1,0,0,0,0
4,68,0,0,0,0,2,0,2,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,120,70,0,0,0,...,0,0,0,0,0,0,0,0,1,3.9,0,146.0,0.75,0.37,0.0,0.0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1
5,60,1,0,0,0,2,0,3,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,190,100,160,90,0,0,0,...,0,0,0,0,0,0,0,0,1,3.5,0,132.0,0.45,0.22,0.0,8.3,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
